# Altering variational equation

This tutorial was prepared by Alexandre Olender, using code made by Dr. Thiago Dias dos Santos. If you have any questions, please email: olender@usp.br

This tutorial is specifically tailored for developers of Spyro, an open-source Python library for modeling waves. Spyro provides a user-friendly interface built on top of Firedrake, making working with complex mathematical models easier. Before you begin this tutorial, it is recommended that you familiarize yourself either with Firedrake (https://www.firedrakeproject.org/documentation.html) or FEniCS (which uses the same domain-specific language called Unified Form Language - UFL). Firedrake is an automated system for the solution of partial differential equations using the finite element method (FEM). In addition to the prerequisite knowledge of Firedrake or FEniCS, a solid understanding of Python programming and basic concepts of numerical methods will be beneficial.  We also recommend previous knowleddege of finite element methods.

In [2]:
# Code in this cell enables plotting in the notebook
%matplotlib inline
import spyro

firedrake:WARNING OMP_NUM_THREADS is not set or is set to a value greater than 1, we suggest setting OMP_NUM_THREADS=1 to improve performance


## Wave class

The `Wave` class in spyro provides a base class for any wave propagator.

In [3]:
help(spyro.Wave)

Help on class Wave in module spyro.solvers.wave:

class Wave(spyro.io.model_parameters.Model_parameters)
 |  Wave(dictionary=None, comm=None)
 |  
 |  Base class for wave equation solvers.
 |  
 |  Attributes:
 |  -----------
 |  comm: MPI communicator
 |  
 |  initial_velocity_model: firedrake function
 |      Initial velocity model
 |  function_space: firedrake function space
 |      Function space for the wave equation
 |  current_time: float
 |      Current time of the simulation
 |  solver_parameters: Python object
 |      Contains solver parameters
 |  real_shot_record: firedrake function
 |      Real shot record
 |  wavelet: list of floats
 |      Values at timesteps of wavelet used in the simulation
 |  mesh: firedrake mesh
 |      Mesh used in the simulation (2D or 3D)
 |  mesh_z: symbolic coordinate z of the mesh object
 |  mesh_x: symbolic coordinate x of the mesh object
 |  mesh_y: symbolic coordinate y of the mesh object
 |  sources: Sources object
 |      Contains informa

It is highly recommended that every wave solver inherit this class. Two of the methods present in it are abstract methods. This means that any concrete class derived from it needs to define them. They are the matrix_building and the forward_solve methods. The name itself is counterintuitive since it does not necessarily have to build a matrix. However it is in this method that we will construct our solver operators based on the desired variational equation. Spyro already has an acoustic wave solver, based on a second order in time pressure equation. 

To differentiate and illustrate a solver using a mixed function space we will focus on implementing an isotropic elastic wave equation. The weak formulation used is:

$$
\begin{split}
    \int_\Omega \rho \frac{\partial^2 \mathbf{u}}{\partial t^2} \cdot \mathbf{v} \,d\Omega
    &=
    \int_{\Gamma}\left(\boldsymbol{\sigma}\mathbf{n}\right) \cdot \mathbf{v} \, d\Gamma\\
    &-
    \int_\Omega \lambda\, \mathrm{tr}\left(\boldsymbol{\epsilon}\left(\mathbf{u}\right)\right) \mathrm{tr}\left(\boldsymbol{\epsilon}\left(\mathbf{v}\right)\right)
    +2\mu\, \boldsymbol{\epsilon}\left(\mathbf{u}\right) :\boldsymbol{\epsilon}\left(\mathbf{v}\right)
     \, d\Omega \\
    &+\int_\Omega\mathbf{f}\cdot \mathbf{v} \, d\Omega.
\end{split}
$$



where $\mathbf{u}=\mathbf{u}\left(\mathbf{x},t\right)$ is the displacement vector corresponding to the elastic waves propagation, $\rho=\rho(\mathbf{x})$ is the density, $\mathbf{f}=\mathbf{f}\left(\mathbf{x},t\right)$ is the body force, $\boldsymbol{\sigma}=\boldsymbol{\sigma}\left(\mathbf{x}\right)$ is the stress tensor, $\lambda=\lambda\left(\mathbf{x}\right)$ is the 1st Lambe parameter, $\mu=\mu\left(\mathbf{x}\right)$ is the 2nd Lambe parameter, and $\boldsymbol{\epsilon}$ is the infinitesimal strain tensor.